### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
#print first rows of school_data_complete
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#Store total number of schools
total_schools = school_data_complete["school_name"].unique()
total_schools = len(total_schools)
total_schools

15

In [4]:
#Store total number of students
total_student = school_data_complete["Student ID"].unique()
total_student = len(total_student)
total_student

39170

In [5]:
#Store total budget
total_budget = school_data_complete["budget"].unique()
total_budget = total_budget.sum()
total_budget

24649428

In [6]:
#Store average math score
math_score = school_data_complete["math_score"].mean()
math_score

78.98537145774827

In [7]:
#Store average reading score
reading_score = school_data_complete["reading_score"].mean()
reading_score

81.87784018381414

In [8]:
#Store overall passing rate
overall_score = (math_score + reading_score)/2
overall_score

80.43160582078121

In [9]:
#Percent students with passing math score
math_over_70 = school_data_complete.loc[school_data_complete["math_score"]>=70,:]
math_over_70 = math_over_70["math_score"].value_counts()
math_over_70 = math_over_70.sum()
percent_passing_math = (math_over_70/total_student)*100
percent_passing_math

74.9808526933878

In [10]:
#Percent students with passing reading score
reading_over_70 = school_data_complete.loc[school_data_complete["reading_score"]>=70,:]
reading_over_70 = reading_over_70["reading_score"].value_counts()
reading_over_70 = reading_over_70.sum()
percent_passing_reading = (reading_over_70/total_student)*100
percent_passing_reading

85.80546336482001

In [11]:
#Create District Summary Dataframe
DistrictSummary = pd.DataFrame(
    {"Summary Statistic":["Number of Schools","Number of Students","Annual Budget","Average Math School","Average Reading Score","Overall Pass Rate","Percent Students Pass Math","Percent Students Pass Reading"],
                                "Value":[total_schools,total_student,total_budget,math_score,reading_score,overall_score,percent_passing_math,percent_passing_reading]
                               }
                            )
DistrictSummary = DistrictSummary.set_index("Summary Statistic")
DistrictSummary

,Value
Summary Statistic,
Number of Schools,1.500000e+01
Number of Students,3.917000e+04
Annual Budget,2.464943e+07
Average Math School,7.898537e+01
Average Reading Score,8.187784e+01
Overall Pass Rate,8.043161e+01
Percent Students Pass Math,7.498085e+01
Percent Students Pass Reading,8.580546e+01


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [12]:
#Isolate school types
school_type = school_data_complete[["school_name","type"]].drop_duplicates()

In [13]:
#Isolate school size
school_size = school_data_complete[["school_name","size"]].drop_duplicates()

In [14]:
#Isolate school budget
school_budget = school_data_complete[["school_name","budget"]].drop_duplicates()

In [15]:
#Merge dataframe to get school_summary df
school_summary = pd.merge(school_type, school_size, on = "school_name")
school_summary = pd.merge(school_summary, school_budget, on = "school_name")

In [16]:
#Calculate per student budget
school_summary["Per Student Budget "] = school_summary["budget"]/school_summary["size"]

In [17]:
#Add average math score to school_summary
group_school = school_data_complete.groupby("school_name")
avg_math = pd.DataFrame(group_school["math_score"].mean())
avg_math = avg_math.reset_index()
school_summary = pd.merge(school_summary, avg_math, on ="school_name")

In [18]:
#Add average reading score to school_summary
avg_reading = pd.DataFrame(group_school["reading_score"].mean())
avg_reading = avg_reading.reset_index()
avg_reading
school_summary = pd.merge(school_summary, avg_reading, on = "school_name")

In [19]:
#Calculate reading passing percentage
reading_over_70 = school_data_complete.loc[school_data_complete["reading_score"]>=70,:]
reading_groupby = reading_over_70.groupby("school_name")
reading_groupby = pd.DataFrame(reading_groupby["reading_score"].count()).reset_index()
reading_groupby.columns = ["school_name","passing_reading"]
school_summary = pd.merge(school_summary, reading_groupby, on="school_name")
school_summary["percent_pass_reading"] = (school_summary["passing_reading"]/school_summary["size"])*100
del school_summary["passing_reading"]

In [20]:
#Calculate math passing percentage
math_over_70 = school_data_complete.loc[school_data_complete["math_score"]>=70,:]
math_groupby = math_over_70.groupby("school_name")
math_groupby = pd.DataFrame(math_groupby["math_score"].count()).reset_index()
math_groupby.columns = ["school_name","passing_math"]
school_summary = pd.merge(school_summary, math_groupby, on="school_name")
school_summary["percent_pass_math"] = (school_summary["passing_math"]/school_summary["size"])*100
del school_summary["passing_math"]

In [21]:
#Calculate Overall Pass Rate
school_summary["Overall Pass Rate"] = (school_summary["percent_pass_reading"] + school_summary["percent_pass_math"])/2

In [22]:
#Rename Columns
school_summary.columns = ["School Name","Type","Size","Annual Budget","Per Student Budget","Average Math Score","Average Reading Score","Percent Passing Reading (>70%)","Percent Passing Math (>70%)","Overall Pass Rate (%)"]
school_summary

,School Name,Type,Size,Annual Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Reading (>70%),Percent Passing Math (>70%),Overall Pass Rate (%)
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,81.316421,65.683922,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,80.739234,65.988471,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,95.854628,93.867121,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,80.862999,66.752967,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,96.539641,93.867718,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,97.039828,94.133477,95.586652
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,81.933280,66.680064,74.306672
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,96.252927,92.505855,94.379391
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,95.945946,94.594595,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [23]:
#Sort school summary df
school_summary = school_summary.sort_values("Overall Pass Rate (%)", ascending=False)
school_summary.head()

,School Name,Type,Size,Annual Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Reading (>70%),Percent Passing Math (>70%),Overall Pass Rate (%)
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,97.039828,94.133477,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,97.308869,93.272171,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,95.945946,94.594595,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,96.539641,93.867718,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [24]:
#Re-sort school summary df
school_summary = school_summary.sort_values("Overall Pass Rate (%)")
school_summary.head()

,School Name,Type,Size,Annual Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Reading (>70%),Percent Passing Math (>70%),Overall Pass Rate (%)
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,80.220055,66.366592,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,80.739234,65.988471,73.363852
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,81.316421,65.683922,73.500171
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,81.222432,66.057551,73.639992
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,79.299014,68.309602,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [25]:
freshman = school_data_complete.loc[school_data_complete["grade"]=="9th",["school_name","math_score"]]
freshman_math_group = freshman.groupby("school_name")
freshman_math = pd.DataFrame(freshman_math_group["math_score"].mean())
freshman_math = freshman_math.reset_index()
freshman_math.columns = ["School","9th"]


In [26]:
sophomore = school_data_complete.loc[school_data_complete["grade"]=="10th",["school_name","math_score"]]
sophomore_math_group = sophomore.groupby("school_name")
sophomore_math = pd.DataFrame(sophomore_math_group["math_score"].mean())
sophomore_math = sophomore_math.reset_index()
sophomore_math.columns = ["School","10th"]


In [27]:
junior = school_data_complete.loc[school_data_complete["grade"]=="11th",["school_name","math_score"]]
junior_math_group = junior.groupby("school_name")
junior_math = pd.DataFrame(junior_math_group["math_score"].mean())
junior_math = junior_math.reset_index()
junior_math.columns = ["School","11th"]

In [28]:
senior = school_data_complete.loc[school_data_complete["grade"]=="12th",["school_name","math_score"]]
senior_math_group = senior.groupby("school_name")
senior_math = pd.DataFrame(senior_math_group["math_score"].mean())
senior_math = senior_math.reset_index()
senior_math.columns = ["School","12th"]

In [29]:
MathScoreByGrade = pd.merge(freshman_math, sophomore_math, on= "School")
MathScoreByGrade = pd.merge(MathScoreByGrade, junior_math, on="School")
MathScoreByGrade = pd.merge(MathScoreByGrade, senior_math, on="School")
MathScoreByGrade

,School,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [30]:
freshman = school_data_complete.loc[school_data_complete["grade"]=="9th",["school_name","reading_score"]]
freshman_reading_group = freshman.groupby("school_name")
freshman_reading = pd.DataFrame(freshman_reading_group["reading_score"].mean())
freshman_reading = freshman_reading.reset_index()
freshman_reading.columns = ["School","9th"]

In [31]:
sophomore = school_data_complete.loc[school_data_complete["grade"]=="10th",["school_name","reading_score"]]
sophomore_reading_group = sophomore.groupby("school_name")
sophomore_reading = pd.DataFrame(sophomore_reading_group["reading_score"].mean())
sophomore_reading = sophomore_reading.reset_index()
sophomore_reading.columns = ["School","10th"]

In [32]:
junior = school_data_complete.loc[school_data_complete["grade"]=="11th",["school_name","reading_score"]]
junior_reading_group = junior.groupby("school_name")
junior_reading = pd.DataFrame(junior_reading_group["reading_score"].mean())
junior_reading = junior_reading.reset_index()
junior_reading.columns = ["School","11th"]

In [33]:
senior = school_data_complete.loc[school_data_complete["grade"]=="12th",["school_name","reading_score"]]
senior_reading_group = senior.groupby("school_name")
senior_reading = pd.DataFrame(senior_reading_group["reading_score"].mean())
senior_reading = senior_reading.reset_index()
senior_reading.columns = ["School","12th"]

In [34]:
ReadingScoreByGrade = pd.merge(freshman_reading, sophomore_reading, on= "School")
ReadingScoreByGrade = pd.merge(ReadingScoreByGrade, junior_reading, on="School")
ReadingScoreByGrade = pd.merge(ReadingScoreByGrade, senior_reading, on="School")
ReadingScoreByGrade

,School,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [35]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [36]:
school_summary1=school_summary[["Per Student Budget","Average Math Score","Average Reading Score","Percent Passing Math (>70%)","Percent Passing Reading (>70%)","Overall Pass Rate (%)"]]

school_summary1["School Spend Category"] = pd.cut(school_summary1["Per Student Budget"],spending_bins,labels=group_names)
school_summary1 = school_summary1.groupby("School Spend Category")
school_summary1.mean()

C:\Users\mkoza\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math (>70%),Percent Passing Reading (>70%),Overall Pass Rate (%)
School Spend Category,,,,,,
<$585,581.000000,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,604.500000,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,635.166667,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,652.333333,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [37]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [38]:
#Binning by School Size
school_summary1=school_summary[["Size","Average Math Score","Average Reading Score","Percent Passing Math (>70%)","Percent Passing Reading (>70%)","Overall Pass Rate (%)"]]
school_summary1["School Size Category"] = pd.cut(school_summary1["Size"],size_bins,labels=group_names)
school_summary1 = school_summary1.groupby("School Size Category")
school_summary1.mean()

C:\Users\mkoza\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Size,Average Math Score,Average Reading Score,Percent Passing Math (>70%),Percent Passing Reading (>70%),Overall Pass Rate (%)
School Size Category,,,,,,
Small (<1000),694.500,83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),1704.400,83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),3657.375,77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [39]:
#Trick question. You can't perform binning on a categorical variable. So I'll just use groupby()
school_summary1=school_summary[["Type","Average Math Score","Average Reading Score","Percent Passing Math (>70%)","Percent Passing Reading (>70%)","Overall Pass Rate (%)"]]
school_summary1 = school_summary1.groupby("Type")
school_summary1.mean() 


,Average Math Score,Average Reading Score,Percent Passing Math (>70%),Percent Passing Reading (>70%),Overall Pass Rate (%)
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


## Analysis

* The analysis presented above based on student test scores suggests several variables may correlate to overall student pass rates. More rigorous statistical analysis, however, should be pursued before taking action at the district or school levels. 
    * Firstly, charter schools appear to perform better on exams than district schools. For example, the top 5 performing schools by overall score were charters, while the lowest 5 performing schools were district schools. Overall pass rate by school type appears to corroborate the finding that charters perform better on exams.  
    * Secondly, based on Scores by Spending and Scores by Size, it would appear that decreasing school size and decreasing spending both increase scores.
    
The first insight mentioned above might lead one to conclude that charters lead to better outcomes for students. However, we must keep in mind that charters can set entry requirements while disctricts cannot. This allows high achieving students to self select out of district schools. Unlike charters, district schools have to serve anyone who walks through the door. The unequal distribution of high achieving students may contribute to higher and lower test scores at charter and district schools, respectively. 

The second insight is an example of how correlation and causation are not the same. It would make sense that decreasing school size would better allow teacher to instruct and meet student needs (e.g. causation). It is unlikely the increase student spending directly reduces overall score (e.g. correlation). 

